In [1]:
# !pip install yfinance

In [18]:
from urllib import request
import yfinance as yf
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing, model_selection, svm
from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px
from plotly.graph_objs import Scatter
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import datetime as dt
import numpy as np

from tensorflow import keras
from sklearn import preprocessing
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [19]:
data = yf.download(
        
    # passes the ticker
    tickers=['AAPL', 'AMZN', 'QCOM', 'META', 'NVDA', 'JPM'],

    group_by = 'ticker',

    threads=True, # Set thread value to true

    # used for access data[ticker]
    period='12mo', 
    interval='1d'

)

data.reset_index(level=0, inplace=True)



fig_left = go.Figure()
fig_left.add_trace(
            go.Scatter(x=data['Date'], y=data['AAPL']['Adj Close'], name="AAPL")
        )
fig_left.add_trace(
            go.Scatter(x=data['Date'], y=data['AMZN']['Adj Close'], name="AMZN")
        )
fig_left.add_trace(
            go.Scatter(x=data['Date'], y=data['QCOM']['Adj Close'], name="QCOM")
        )
fig_left.add_trace(
            go.Scatter(x=data['Date'], y=data['META']['Adj Close'], name="META")
        )
fig_left.add_trace(
            go.Scatter(x=data['Date'], y=data['NVDA']['Adj Close'], name="NVDA")
        )
fig_left.add_trace(
            go.Scatter(x=data['Date'], y=data['JPM']['Adj Close'], name="JPM")
        )
fig_left.update_layout(paper_bgcolor="#14151b", plot_bgcolor="#14151b", font_color="white")

plot_div_left = plot(fig_left, auto_open=False, output_type='div')

[*********************100%%**********************]  6 of 6 completed


In [20]:
data

Ticker       Date        META                                                  \
Price                    Open        High         Low       Close   Adj Close   
0      2023-03-23  202.839996  207.880005  202.149994  204.279999  204.063492   
1      2023-03-24  205.179993  207.580002  203.550003  206.009995  205.791641   
2      2023-03-27  204.809998  205.860001  201.360001  202.839996  202.625015   
3      2023-03-28  200.149994  201.029999  197.899994  200.679993  200.467300   
4      2023-03-29  203.559998  205.720001  202.539993  205.350006  205.132355   
..            ...         ...         ...         ...         ...         ...   
247    2024-03-18  491.910004  497.420013  486.809998  496.980011  496.980011   
248    2024-03-19  488.170013  496.630005  481.279999  496.239990  496.239990   
249    2024-03-20  499.500000  508.200012  495.170013  505.519989  505.519989   
250    2024-03-21  514.710022  515.039978  506.010010  507.760010  507.760010   
251    2024-03-22  507.000000  509.970001  504.339996  509.579987  509.579987   

Ticker                   JPM                          ...        AMZN  \
Price     Volume        Open        High         Low  ...         Low   
0       27389700  127.900002  129.529999  126.019997  ...   97.620003   
1       27716400  125.629997  125.680000  123.110001  ...   96.400002   
2       18527200  126.790001  129.380005  126.290001  ...   97.080002   
3       19127300  128.710007  129.339996  127.739998  ...   96.290001   
4       18851100  130.100006  130.220001  127.769997  ...   98.559998   
..           ...         ...         ...         ...  ...         ...   
247     11755300  190.630005  192.699997  189.889999  ...  174.279999   
248     10903100  192.710007  193.929993  192.380005  ...  173.520004   
249     11711100  194.190002  196.559998  193.610001  ...  174.639999   
250      9712500  196.520004  199.429993  196.330002  ...  178.149994   
251      8117000  199.009995  200.479996  196.539993  ...  176.750000   

Ticker                                          NVDA                          \
Price        Close   Adj Close    Volume        Open        High         Low   
0        98.709999   98.709999  57559300  271.149994  274.989990  266.899994   
1        98.129997   98.129997  56095400  270.309998  271.670013  263.549988   
2        98.040001   98.040001  46721300  268.369995  270.000000  263.649994   
3        97.239998   97.239998  38720100  264.470001  265.130005  258.500000   
4       100.250000  100.250000  49783300  268.250000  270.779999  265.970001   
..             ...         ...       ...         ...         ...         ...   
247     174.479996  174.479996  31250700  903.880005  924.049988  870.849976   
248     175.899994  175.899994  26880900  867.000000  905.440002  850.099976   
249     178.149994  178.149994  29947200  897.969971  904.099976  882.229980   
250     178.149994  178.149994  32824300  923.000000  926.479980  904.049988   
251     178.869995  178.869995  27964100  911.409973  947.780029  908.340027   

Ticker                                    
Price        Close   Adj Close    Volume  
0       271.910004  271.822815  56488900  
1       267.790009  267.704132  45492000  
2       265.309998  265.224945  36102600  
3       264.100006  264.015320  35610400  
4       269.839996  269.753479  39369400  
..             ...         ...       ...  
247     884.549988  884.549988  66897600  
248     893.979980  893.979980  67217100  
249     903.719971  903.719971  47906300  
250     914.349976  914.349976  48037200  
251     942.890015  942.890015  58521500  

[252 rows x 37 columns]

In [21]:
# function to calculate percentage difference considering baseValue as 100%
def percentageChange(baseValue, currentValue):
    return((float(currentValue)-baseValue) / abs(baseValue)) *100.00

# function to get the actual value using baseValue and percentage
def reversePercentageChange(baseValue, percentage):
    return float(baseValue) + float(baseValue * percentage / 100.00)

# function to transform a list of values into the list of percentages. For calculating percentages for each element in the list
# the base is always the previous element in the list.
def transformToPercentageChange(x):
    baseValue = x[0]
    x[0] = 0
    for i in range(1,len(x)):
        pChange = percentageChange(baseValue,x[i])
        baseValue = x[i]
        x[i] = pChange

# function to transform a list of percentages to the list of actual values. For calculating actual values for each element in the list
# the base is always the previous calculated element in the list.

dictionaryofdateandprice={}

def reverseTransformToPercentageChange(baseValue, x):
    x_transform = []
    for i in range(0,len(x)):
        value = reversePercentageChange(baseValue,x[i])
        baseValue = value
        x_transform.append(value)
    return x_transform

In [22]:
def datapred(ticker_name, number_of_days):
    df = yf.download(tickers=ticker_name,group_by = 'ticker',threads=True,period='12mo',interval='1d')
    df.reset_index(level=0, inplace=True)
    
    baseValue = df['Close'][0]

    # create a new dataframe which is then transformed into relative percentages
    data = df.sort_index(ascending=True, axis=0)
    new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])
    for i in range(0,len(data)):
        new_data['Date'][i] = data['Date'][i]
        new_data['Close'][i] = data['Close'][i]

    # transform the 'Close' series into relative percentages
    transformToPercentageChange(new_data['Close'])

    # set Dat column as the index
    new_data.index = new_data.Date
    new_data.drop('Date', axis=1, inplace=True)

    # create train and test sets
    dataset = new_data[0:1500].values
    print("====dataset====")
    print(dataset)
    print("====len dataset====")
    print(len(dataset))
    train, valid = train_test_split(dataset, train_size=0.99, test_size=0.01, shuffle=False)

    # convert dataset into x_train and y_train.
    # prediction_window_size is the size of days windows which will be considered for predicting a future value.

    prediction_window_size = 60
    x_train, y_train = [], []
    for i in range(prediction_window_size,len(train)):
        x_train.append(dataset[i-prediction_window_size:i,0])
        y_train.append(dataset[i,0])
    x_train, y_train = np.array(x_train).astype(np.float32), np.array(y_train).astype(np.float32)
    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))


    # X_train = []
    # y_train = []
    # for i in range(60, 1500):
    #     X_train.append(np.array(dataset[60:1600].astype(np.float32))[i-60:i])
    #     y_train.append(np.array(dataset[60:1600].astype(np.float32))[i])
    # X_train, y_train = np.array(X_train), np.array(y_train)
    # X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))



    x_valid, y_valid = [], []
    for i in range(60,120):
        x_valid.append(dataset[i-prediction_window_size:i,0])
        y_valid.append(dataset[i,0])
        
    X_test = np.asarray(x_valid).astype('float32')
    #y_test = np.asarray(y_valid).astype('float32')

    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))


    ##################################################################################################
    # create and fit the LSTM network
    # Initialising the RNN
    model = Sequential()
    # Adding the first LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
    model.add(Dropout(0.2))

    # Adding a second LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))

    # Adding a third LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))

    # Adding a fourth LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))

    # Adding the output layer
    model.add(Dense(units = 1))
    # Compiling the RNN
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    # accu = model.evaluate(x_test,y_test)
    # print("accuracy is")
    # print(acu)
    # Fitting the RNN to the Training set
    model.fit(x_train, y_train, epochs = 20, batch_size = 1000)
  

    df_ml = df[['Adj Close']]
    forecast_out = int(number_of_days)
    df_ml['Prediction'] = df[['Adj Close']].shift(-forecast_out)
    # Splitting data for Test and Train
    X = np.array(df_ml.drop(['Prediction'],1))
    X = preprocessing.scale(X)
    X_forecast = X[-forecast_out:]
    X = X[:-forecast_out]
    y = np.array(df_ml['Prediction'])
    y = y[:-forecast_out]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

    X_forecast = np.array(X_forecast).astype(np.float32)
    X_forecast = np.reshape(X_forecast, (X_forecast.shape[0],X_forecast.shape[1],1))

    forecast_prediction = model.predict(X_forecast)
    forecast = forecast_prediction.tolist()
    print(forecast_prediction)

    pred_dict = {"Date": [], "Prediction": []}
    for i in range(0, len(forecast)):
        pred_dict["Date"].append(dt.datetime.today() + dt.timedelta(days=i))
        pred_dict["Prediction"].append(forecast[i][0])
        
    pred_df = pd.DataFrame(pred_dict)
        
    pred_fig = go.Figure([go.Scatter(x=pred_df['Date'], y=pred_df['Prediction'])])
    pred_fig.update_xaxes(rangeslider_visible=True)
    pred_fig.update_layout(paper_bgcolor="#14151b", plot_bgcolor="#14151b", font_color="white")
    plot_div_pred = plot(pred_fig, auto_open=False, output_type='div')
    
    return pred_df

In [23]:
ticker_name = 'AMZN'
number_of_days = 30
pred_df = datapred(ticker_name, number_of_days)
pred_df

[*********************100%%**********************]  1 of 1 completed


====dataset====
[[0]
 [-0.587581639584802]
 [-0.0917113425145748]
 [-0.815996577200266]
 [3.0954362426533533]
 [1.7456359102244388]
 [1.2647067799287683]
 [-0.8519675143943976]
 [1.5037527888523978]
 [-2.7417013542964686]
 [0.9495540049077156]
 [0.10778033801971233]
 [-2.2022120390757625]
 [-2.091669711959809]
 [4.67136830142994]
 [0.10742246944572822]
 [0.22436418178336404]
 [-0.42826048390145]
 [1.9550341547772163]
 [-0.4698039106680585]
 [3.034391291745232]
 [-0.7011967150520266]
 [-3.427172037496596]
 [2.3496184744855624]
 [4.610398345515921]
 [-3.9792412663683407]
 [-3.2242712137328815]
 [1.548254928379388]
 [0.019303553981593596]
 [0.33767339022538034]
 [1.5961573674128606]
 [0.16089169321718977]
 [0.7464810562778676]
 [3.3483395262233415]
 [1.80596952507349]
 [-1.7115333960796282]
 [0.8525256609829547]
 [1.9784214370623647]
 [1.8518504813616383]
 [2.2943736154795724]
 [-1.6081265352017287]
 [-1.066664829049059]
 [-0.017393506729303634]
 [1.5305697616548972]
 [-1.4989293361884368

C:\Users\shubh\AppData\Local\Temp\ipykernel_14116\813137417.py:96: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\shubh\AppData\Local\Temp\ipykernel_14116\813137417.py:98: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



[[0.02245828]
 [0.02271188]
 [0.02316038]
 [0.02287619]
 [0.02301927]
 [0.02305451]
 [0.0233508 ]
 [0.02316647]
 [0.02244271]
 [0.02239365]
 [0.02242475]
 [0.02256744]
 [0.02261312]
 [0.02218261]
 [0.02200951]
 [0.02208528]
 [0.02249783]
 [0.02257105]
 [0.02217548]
 [0.02235062]
 [0.02275772]
 [0.02234584]
 [0.0222064 ]
 [0.02194689]
 [0.02246187]
 [0.02245468]
 [0.02228499]
 [0.02201779]
 [0.02201779]
 [0.02193273]]


,Date,Prediction
0,2024-03-25 14:16:19.834968,0.022458
1,2024-03-26 14:16:19.834968,0.022712
2,2024-03-27 14:16:19.834968,0.023160
3,2024-03-28 14:16:19.834968,0.022876
4,2024-03-29 14:16:19.834968,0.023019
5,2024-03-30 14:16:19.834968,0.023055
6,2024-03-31 14:16:19.834968,0.023351
7,2024-04-01 14:16:19.834968,0.023166
8,2024-04-02 14:16:19.834968,0.022443
9,2024-04-03 14:16:19.834968,0.022394


In [24]:
dff = pd.read_csv("Fortune_1000.csv")
dff

,company,rank,rank_change,revenue,profit,num. of employees,sector,city,state,newcomer,ceo_founder,ceo_woman,profitable,prev_rank,CEO,Website,Ticker,Market Cap
0,Walmart,1,0,572754.0,13673.0,2300000.0,Retailing,Bentonville,AR,no,no,no,yes,1,C. Douglas McMillon,https://www.stock.walmart.com,WMT,352037
1,Amazon,2,0,469822.0,33364.0,1608000.0,Retailing,Seattle,WA,no,no,no,yes,2,Andrew R. Jassy,www.amazon.com,AMZN,1202717
2,Apple,3,0,365817.0,94680.0,154000.0,Technology,Cupertino,CA,no,no,no,yes,3,Timothy D. Cook,www.apple.com,AAPL,2443962
3,CVS Health,4,0,292111.0,7910.0,258000.0,Health Care,Woonsocket,RI,no,no,yes,yes,4,Karen Lynch,https://www.cvshealth.com,CVS,125204
4,UnitedHealth Group,5,0,287597.0,17285.0,350000.0,Health Care,Minnetonka,MN,no,no,no,yes,5,Andrew P. Witty,www.unitedhealthgroup.com,UNH,500468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,1-800-Flowers.com,997,0,2122.2,118.7,4800.0,Retailing,Jericho,NY,no,no,no,yes,,Christopher G. McCann,https://www.1800flowers.com,FLWS,830
997,Cowen,998,0,2112.8,295.6,1534.0,Financials,New York,NY,no,no,no,yes,,Jeffrey Solomon,https://www.cowen.com,COWN,1078
998,Ashland,999,0,2111.0,220.0,4100.0,Chemicals,Wilmington,DE,no,no,no,yes,,Guillermo Novo,https://www.ashland.com,ASH,5601.9
999,DocuSign,1000,0,2107.2,-70.0,7461.0,Technology,San Francisco,CA,no,no,no,no,,Allan C. Thygesen,https://www.docusign.com,DOCU,21302.8


In [25]:
cmp_ticker = dff[dff["company"]=="CVS Health"]
cmp_ticker["Ticker"].values[0]

'CVS'